## Unit-Resource RCPSP with Calendars and Blocking Time

This notebook demonstrates how to model and solve a Resource-Constrained Project Scheduling Problem with unit resources (workers/machines), resource calendars, and blocking time during breaks. The problem extends the classical RCPSP by incorporating:
- **Unit resources**: Each resource is a single worker or machine (unit capacity)
- **Resource calendars**: Resources have availability functions defining working periods and breaks
- **Blocking time**: Resources remain blocked during their breaks if assigned to a task
- **Multi-mode assignment**: Each task can be processed by any resource from its eligible set
- **Non-preemptive processing**: Once a resource starts a task, it must complete it

The model uses IBM's CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.

### Problem Definition

Consider a finite set of tasks indexed by $i \in [1..N]$. We have a set of unit resources (workers/machines) indexed by $r \in [1..R]$. Each resource $r$ has a defined availability function (calendar) $F_r(t) \in \{0,1\}$ that specifies when the resource is working (1) and when it has a break (0), as described in model 3.2.

For each task $i$, there exists a set of eligible resources (modes) $M[i] \subseteq [1..R]$. For every pair $(i,r)$ where $r \in M[i]$, a net processing time $PT_{ir} > 0$ is given.

**Key properties:**

1. **Unit Blocking**: If task $i$ is assigned to resource $r$, this resource is blocked for the entire duration of the interval (including breaks given by the calendar), which corresponds to the NOOVERLAP condition from model 3.4.

2. **Non-preemptive Assignment**: The task must be completed by the resource that started it. This is ensured by the structure of the interval variable $y_{ir}$.

3. **Calendars**: Work does not proceed during non-working time (e.g., weekends), but the interval remains active. This is modeled using the `intensity` or `forbidExtent` attribute.

A set of precedence relations $P \subseteq [1..N]\times[1..N]$ specifies the required order of tasks. The objective is to minimize the makespan $C_{\max}$, i.e., the completion time of the last finishing task.

### CP Optimizer Formulation

$$
\begin{aligned}
\min \quad 
& \max_{i \in [1..N]} \mathrm{endOf}(x_i) 
\qquad &\qquad & \text{(1)} \\[2mm]
\text{s.t.} \quad
& \mathrm{alternative}\!\left(x_i,\; [\,y_{ir}\,]_{r\in M[i]}\right), 
\qquad & \forall i\in[1..N] 
\quad & \text{(2)} \\[1mm]
& \mathrm{noOverlap}\!\left(\{y_{ir} \mid i\in[1..N], r\in M[i]\},\, \Delta_{\mathrm{setup}}\right), 
\qquad & \forall r\in[1..R] 
\quad & \text{(3)} \\[1mm]
& \mathrm{endBeforeStart}(x_i, x_j), 
\qquad & \forall (i,j)\in P 
\quad & \text{(4)} \\[1mm]
& \text{interval } x_i, 
\qquad & \forall i\in[1..N] 
\quad & \text{(5)} \\[1mm]
& \text{interval } y_{ir}\ \text{optional}, 
\qquad & \forall i\in[1..N],\ \forall r\in M[i] 
\quad & \text{(6)} \\[1mm]
& \text{where:} \qquad \mathrm{sizeOf}(y_{ir}) = PT_{ir}, 
\qquad & \forall i\in[1..N],\ \forall r\in M[i] 
\quad & \text{(9a)} \\[1mm]
& \phantom{\text{where:}} \qquad \mathrm{intensity}(y_{ir}) = F_r, 
\qquad & \forall i\in[1..N],\ \forall r\in M[i] 
\quad & \text{(9b)}
\end{aligned}
$$

**Variables:**
- $x_i$: Interval variable representing task $i$
- $y_{ir}$: Optional interval variable representing the execution of task $i$ by resource $r$, where:
  - **(9a)** The net work content (size) equals $PT_{ir}$
  - **(9b)** The intensity function follows the resource calendar $F_r$

### Constraint Explanations

**Objective:**
- **(1)** Minimize the makespan $C_{\max} = \max_{i\in[1..N]} \mathrm{endOf}(x_i)$ — the completion time of the last finishing task.

**Constraints:**

- **(2) Resource Selection (Alternative):** This constraint enforces that for each task $x_i$, exactly one resource $r$ from the set $M[i]$ will be selected. The interval $x_i$ is synchronized with the chosen $y_{ir}$. This ensures the requirement that "one resource starts and the same one completes" the task.

- **(3) Resource Blocking (NoOverlap):** This replaces the capacity constraints from traditional RCPSP formulations. Since we work with unit resources (individual workers/machines), we use a global `noOverlap` constraint on the set of all intervals assigned to resource $r$.
  
  This addresses the "blocking time" requirement. Even if a resource (e.g., worker Bob) has a break during the weekend (due to the `intensity` function), the interval $y_{ir}$ still exists and spans the weekend. The `noOverlap` constraint ensures that no other task can be inserted into this interval (not even during the weekend break).
  
  $\Delta_{\mathrm{setup}}$ represents an optional matrix of setup times between task families.

- **(4) Precedence Relations:** Standard precedence constraints ensuring that task $i$ must complete before task $j$ can start for all $(i,j) \in P$.

- **(5) Task Intervals:** Declares interval variables for each task.

- **(6) Optional Resource Intervals:** Declares optional interval variables for each task-resource pair. Only the selected resource interval will be present in the solution.

- **(9a) Net Processing Time (Size):** Defines that the task requires $PT_{ir}$ units of net working time when executed by resource $r$. This is the actual work content that must be completed.

- **(9b) Resource Calendars (Intensity):** Applies the availability function $F_r$. When $F_r(t) = 0$ (e.g., weekend), work stops, the "size" does not decrease, and the interval extends in time ("length" increases) until the total "size" is satisfied.

**Key Modeling Insights:**

- The combination of `intensity` and `noOverlap` ensures both calendar-aware scheduling and blocking during breaks.
- The `alternative` constraint naturally handles the multi-mode aspect while ensuring non-preemptive assignment.
- The model distinguishes between:
  - **Size** (net work content): How much actual work needs to be done
  - **Length** (calendar time): How much time the task spans, including breaks

When a resource has a break (calendar value = 0), the interval continues to exist and blocks the resource, but no work progress is made (size doesn't decrease). Once the resource returns to work (calendar value = 1), work resumes until the full size is completed.

### Code Example

First, we import the necessary libraries:

In [6]:
from docplex.cp.model import CpoModel, CpoStepFunction
from docplex.cp.utils import compare_natural
import docplex.cp.utils_visu as visu
import matplotlib.pyplot as plt

Define problem data with:
- Tasks and their eligible resources
- Processing times for each task-resource combination
- Precedence relations
- Resource calendars (working days vs. breaks)

In [7]:
# Problem parameters
N = 8  # Number of tasks (including dummy start and end)
R = 3  # Number of resources (workers)

# Eligible resources for each task (M[i] = list of resource indices)
M = {
    0: [0],        # Dummy start - any resource
    1: [0, 1],     # Task 1 can be done by resource 0 or 1
    2: [1, 2],     # Task 2 can be done by resource 1 or 2
    3: [0, 2],     # Task 3 can be done by resource 0 or 2
    4: [0, 1],     # Task 4 can be done by resource 0 or 1
    5: [1, 2],     # Task 5 can be done by resource 1 or 2
    6: [0, 1, 2],  # Task 6 can be done by any resource
    7: [0]         # Dummy end - any resource
}

# Processing times PT[i][r] for each task-resource pair
PT = {
    (0, 0): 0,
    (1, 0): 3, (1, 1): 4,
    (2, 1): 5, (2, 2): 6,
    (3, 0): 4, (3, 2): 5,
    (4, 0): 6, (4, 1): 7,
    (5, 1): 3, (5, 2): 4,
    (6, 0): 5, (6, 1): 4, (6, 2): 6,
    (7, 0): 0
}

# Precedence relations P = [(i, j)] meaning task i must finish before task j starts
P = [
    (0, 1), (0, 2), (0, 3),  # Start must precede tasks 1, 2, 3
    (1, 4), (2, 4),           # Tasks 1 and 2 must finish before 4
    (3, 5),                   # Task 3 must finish before 5
    (4, 6), (5, 6),           # Tasks 4 and 5 must finish before 6
    (6, 7)                    # Task 6 must finish before end
]

# Resource calendars: F[r] is a step function for resource r
# Format: (start_time, end_time, intensity_value)
# intensity = 1 means working, intensity = 0 means break
# Example: 5-day work week (Mon-Fri work, Sat-Sun break)
WEEK_LENGTH = 7
WORK_DAYS = 5
HORIZON = 50

def create_weekly_calendar(horizon):
    """Create a calendar with 5 working days and 2 days break per week"""
    calendar = []
    t = 0
    while t < horizon:
        # Working period (5 days)
        work_end = min(t + WORK_DAYS, horizon)
        calendar.append((t, work_end, 1))
        t = work_end
        
        # Break period (2 days) - only if we haven't reached horizon
        if t < horizon:
            break_end = min(t + (WEEK_LENGTH - WORK_DAYS), horizon)
            calendar.append((t, break_end, 0))
            t = break_end
    
    return calendar

# Each resource has the same weekly calendar
F = {r: create_weekly_calendar(HORIZON) for r in range(R)}

# Setup times between tasks (optional - set to 0 for now)
SETUP_TIME = 0

print(f"Problem instance: {N} tasks, {R} resources")
print(f"\nResource calendar for resource 0 (first 3 weeks):")
for i, (start, end, intensity) in enumerate(F[0][:6]):
    period_type = "Work" if intensity == 1 else "Break"
    print(f"  [{start:2d}, {end:2d}): {period_type}")

Problem instance: 8 tasks, 3 resources

Resource calendar for resource 0 (first 3 weeks):
  [ 0,  5): Work
  [ 5,  7): Break
  [ 7, 12): Work
  [12, 14): Break
  [14, 19): Work
  [19, 21): Break


Build the CP Optimizer model following the mathematical formulation:

**Note on Calendar Implementation:** The mathematical formulation uses `intensity(y_ir) = F_r` (constraint 9b), which tells the solver to work only during periods where F_r(t) = 1. In CP Optimizer's docplex API, this is implemented using `forbid_extent`, which prevents the interval from overlapping with forbidden time periods (where F_r(t) = 0). Both approaches achieve the same result: the task's "size" (net work) only decreases during working periods, and the interval automatically extends through breaks.

In [8]:
# Create the model
mdl = CpoModel(name="UnitResourceRCPSP_Calendars")

# Build step functions for resource calendars
# A step function with value 100 means "forbidden" (break time)
# A step function with value 0 means "allowed" (working time)
calendar_functions = {}
for r in range(R):
    sf = CpoStepFunction()
    for start, end, intensity in F[r]:
        if intensity == 0:  # Break period - forbid work
            sf.add_value(start, end, 100)
        # If intensity == 1 (working time), we don't add anything (default is 0 = allowed)
    calendar_functions[r] = sf

# (5) Create interval variables for each task x[i]
x = {i: mdl.interval_var(name=f"T{i}") for i in range(N)}

# (6) Create optional interval variables y[i,r] for each task-resource pair
y = {}
for i in range(N):
    for r in M[i]:
        # (9a) Set size to processing time PT[i,r]
        size = PT[(i, r)]
        
        # Create the optional interval
        y[(i, r)] = mdl.interval_var(
            optional=True,
            size=size,
            name=f"T{i}_R{r}"
        )
        
        # (9b) Apply calendar constraint - forbid work during break times
        if len(calendar_functions[r].get_step_list()) > 0:
            mdl.add(mdl.forbid_extent(y[(i, r)], calendar_functions[r]))

# (2) Alternative constraint: each task must choose exactly one resource
for i in range(N):
    mdl.add(mdl.alternative(x[i], [y[(i, r)] for r in M[i]]))

# (3) NoOverlap constraint: each resource can only work on one task at a time
for r in range(R):
    # Collect all intervals assigned to resource r
    resource_intervals = [y[(i, r)] for i in range(N) if r in M[i]]
    if resource_intervals:
        if SETUP_TIME > 0:
            # With setup times
            mdl.add(mdl.no_overlap(resource_intervals, SETUP_TIME))
        else:
            # Without setup times
            mdl.add(mdl.no_overlap(resource_intervals))

# (4) Precedence constraints
for (i, j) in P:
    mdl.add(mdl.end_before_start(x[i], x[j]))

# (1) Objective: minimize makespan (end time of last task)
makespan = mdl.max([mdl.end_of(x[i]) for i in range(N)])
mdl.add(mdl.minimize(makespan))

print("Model created successfully")
print(f"Variables: {len(x)} task intervals, {len(y)} task-resource intervals")
print(f"Constraints: {len(P)} precedence relations, {R} no-overlap constraints")

Model created successfully
Variables: 8 task intervals, 15 task-resource intervals
Constraints: 9 precedence relations, 3 no-overlap constraints


Solve the model:

In [9]:
# Solve the model
print("Solving the model...")
res = mdl.solve(TimeLimit=30, LogVerbosity="Quiet")

if res:
    print("\n" + "="*60)
    print("SOLUTION FOUND")
    print("="*60)
    print(f"Makespan: {res.get_objective_values()[0]:.1f}")
    print(f"Solve time: {res.get_solve_time():.2f}s")
else:
    print("\nNo solution found")

Solving the model...

No solution found


Display the solution details:

In [10]:
if res:
    print("\n" + "="*60)
    print("TASK SCHEDULE")
    print("="*60)
    print(f"{'Task':<6} {'Resource':<10} {'Start':<8} {'End':<8} {'Length':<8} {'Size':<6}")
    print("-"*60)
    
    for i in range(N):
        task_sol = res.get_var_solution(x[i])
        
        # Find which resource was selected
        selected_resource = None
        for r in M[i]:
            if res.get_var_solution(y[(i, r)]).is_present():
                selected_resource = r
                break
        
        if selected_resource is not None:
            resource_sol = res.get_var_solution(y[(i, selected_resource)])
            start = resource_sol.get_start()
            end = resource_sol.get_end()
            length = resource_sol.get_length()
            size = resource_sol.get_size()
            
            task_name = f"T{i}"
            if i == 0:
                task_name = "Start"
            elif i == N-1:
                task_name = "End"
            
            print(f"{task_name:<6} R{selected_resource:<9} {start:<8} {end:<8} {length:<8} {size:<6}")
    
    print("\n" + "="*60)
    print("RESOURCE UTILIZATION")
    print("="*60)
    
    for r in range(R):
        print(f"\nResource {r}:")
        tasks_on_resource = []
        for i in range(N):
            if r in M[i] and res.get_var_solution(y[(i, r)]).is_present():
                task_sol = res.get_var_solution(y[(i, r)])
                tasks_on_resource.append((
                    i,
                    task_sol.get_start(),
                    task_sol.get_end(),
                    task_sol.get_size()
                ))
        
        tasks_on_resource.sort(key=lambda x: x[1])  # Sort by start time
        
        for task_id, start, end, size in tasks_on_resource:
            task_name = f"T{task_id}"
            if task_id == 0:
                task_name = "Start"
            elif task_id == N-1:
                task_name = "End"
            print(f"  {task_name}: [{start}, {end}) - work content: {size} units")

Visualize the schedule with resource calendars:

In [11]:
if res and visu.is_visu_enabled():
    plt.rcParams["figure.figsize"] = (16, 8)
    
    # Panel for each resource showing its schedule
    for r in range(R):
        visu.panel(f"Resource {r}")
        
        # Draw calendar breaks as background
        for start, end, intensity in F[r]:
            if intensity == 0:  # Break period
                visu.pause(start, end)
        
        # Draw tasks assigned to this resource
        for i in range(N):
            if r in M[i] and res.get_var_solution(y[(i, r)]).is_present():
                task_sol = res.get_var_solution(y[(i, r)])
                
                task_name = f"T{i}"
                if i == 0:
                    task_name = "Start"
                elif i == N-1:
                    task_name = "End"
                
                # Add size information to label
                size = task_sol.get_size()
                length = task_sol.get_length()
                label = f"{task_name} (work:{size}, span:{length})"
                
                visu.interval(task_sol, r, label)
    
    visu.show()
else:
    if not res:
        print("No solution to visualize")
    else:
        print("Visualization not available")

No solution to visualize


### Additional Resources

- **Related Problems:**
    - Classical RCPSP: [rcpsp.ipynb](https://github.com/radovluk/CP_Cookbook/blob/main/notebooks/rcpsp.ipynb)
    - Multi-Mode RCPSP: [multimode_rcpsp.ipynb](https://github.com/radovluk/CP_Cookbook/blob/main/notebooks/multimode_rcpsp.ipynb)
- **CP Optimizer Documentation:**
    - [Interval Variables](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.interval_var)
    - [Intensity Functions](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.step_function)
    - [NoOverlap Constraint](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.no_overlap)